### Required Libraries

In [1]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from operator import itemgetter
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders.pdf import PyMuPDFLoader

### Defining the models

In [2]:
Model = "llama3"; 
Embedding_Model = "mxbai-embed-large"
model = Ollama(model=Model)

### Loading data and splitting into chunks

In [3]:
loader = DirectoryLoader(path="testDataMD",glob="*.txt")
documents = loader.load()

text_splitter = CharacterTextSplitter(separator="##", chunk_size=1000, chunk_overlap=0)
docs2 = text_splitter.split_documents(documents)

Created a chunk of size 1161, which is longer than the specified 1000
Created a chunk of size 2033, which is longer than the specified 1000
Created a chunk of size 2603, which is longer than the specified 1000
Created a chunk of size 1782, which is longer than the specified 1000
Created a chunk of size 5099, which is longer than the specified 1000
Created a chunk of size 2110, which is longer than the specified 1000
Created a chunk of size 1398, which is longer than the specified 1000
Created a chunk of size 3672, which is longer than the specified 1000
Created a chunk of size 1463, which is longer than the specified 1000
Created a chunk of size 1654, which is longer than the specified 1000
Created a chunk of size 2628, which is longer than the specified 1000
Created a chunk of size 1706, which is longer than the specified 1000
Created a chunk of size 1773, which is longer than the specified 1000
Created a chunk of size 2232, which is longer than the specified 1000
Created a chunk of s

### Creating the vectorstore

In [4]:
vectors = FAISS.from_documents(docs2, OllamaEmbeddings(model = Model))
retriever2 = vectors.as_retriever()

### Defining the RAG Model

In [18]:
# A function to combine retrieved documents into a string 

def combineDocs(docs):
    context = "\n\n".join(f'Document [ Metadata : \n{doc.metadata} \nPage Content : \n{doc.page_content} ]' for doc in docs)
    return context

def combineDocs2(docs):
    context = "\n\n".join(f'{doc.page_content}'  for doc in docs)
    return context

#### Method 1 (Complicated internet answer)

In [6]:
def create_model(vectorDB):
    
    # This is where the conversation will be stored
    memory = ConversationBufferMemory()

    # Creating a conversation model for ConvChain

    def rag_retrieval(question, memory):

        # Retrieve docs from vector DB
        docs = vectorDB.similarity_search(question, k = 4)
        context = combineDocs2(docs)

        conversation = memory.load_memory_variables({}).get('history', '')

        # Add context to the prompt
        prompt = f'Answer the question based on the context below and the previous conversations. If you cannot\nanswer the question, reply "Oof that\'s a tough one, i don\'t really know this"\n\nPrevious Conversation : \n{conversation}\n\nContext : \n{context}\n\nQuestion from user: {question}'

        response = model.invoke(prompt)

        memory.add_user_message(question)
        memory.add_bot_message(response)

        return response
    
    chain = ConversationChain(memory = memory, conversation_model = rag_retrieval)

    return chain

#### Method 2 (Self)

In [19]:
chatHistory = "\n"

latestPrompt = ""

template = """
You are a personal assistant answering the user's questions. You are answering questions based on the context that user is providing and the conversation history.

Context given by user : \n{context}

Current Conversation : \n
{conversation}
\n
User's Question : {question}

Assistant : 

"""

prompt = PromptTemplate.from_template(template)

def chat(question): 
    
    global chatHistory
    global latestPrompt
    # Retrieve docs from vector DB
    docs = vectors.similarity_search(question, k = 4)
    contextString = combineDocs2(docs)
    
    query = prompt.format(conversation = chatHistory, context = contextString, question = question)
    latestPrompt = query
    response = model.invoke(query)
    
    chatHistory = chatHistory + "\n\nUser : " + question + "\n\nAssistant : " + response

    return response


In [20]:
chat("What is a browser's high level structure?")

'The browser\'s high-level structure consists of several components:\n\n1. **User Interface (UI):** This includes the address bar, back/forward button, bookmarking menu, and other visual elements that users interact with.\n2. **Browser Engine:** This marshals actions between the UI and the rendering engine, ensuring a seamless interaction between these two components.\n3. **Rendering Engine:** Responsible for displaying requested content, including parsing HTML and CSS, and displaying the parsed content on the screen.\n4. **Networking:** Handles network calls such as HTTP requests, using different implementations for different platforms (behind a platform-independent interface).\n5. **UI Backend:** Used for drawing basic widgets like combo boxes and windows, exposing a generic interface that is not platform-specific. This backend uses operating system user interface methods.\n6. **JavaScript Engine:** Interpreter used to parse and execute JavaScript code.\n7. **Data Storage:** A persis

In [21]:
print(chatHistory)



User : What is a browser's high level structure?
Assistant : The browser's high-level structure consists of several components:

1. **User Interface (UI):** This includes the address bar, back/forward button, bookmarking menu, and other visual elements that users interact with.
2. **Browser Engine:** This marshals actions between the UI and the rendering engine, ensuring a seamless interaction between these two components.
3. **Rendering Engine:** Responsible for displaying requested content, including parsing HTML and CSS, and displaying the parsed content on the screen.
4. **Networking:** Handles network calls such as HTTP requests, using different implementations for different platforms (behind a platform-independent interface).
5. **UI Backend:** Used for drawing basic widgets like combo boxes and windows, exposing a generic interface that is not platform-specific. This backend uses operating system user interface methods.
6. **JavaScript Engine:** Interpreter used to parse and e

In [22]:
chat("Can you give me the answer to my previous question as a paragraph?")

"Here is the answer to your previous question in a paragraph:\n\nA browser's high-level structure consists of several components that work together to provide a seamless browsing experience. At the top level, there is the User Interface (UI), which includes the address bar, back/forward button, bookmarking menu, and other visual elements that users interact with. Below this, we have the Browser Engine, which marshals actions between the UI and the rendering engine, ensuring a smooth interaction between these two components. The Rendering Engine is responsible for displaying requested content, including parsing HTML and CSS, and displaying the parsed content on the screen. Networking handles network calls such as HTTP requests, using different implementations for different platforms. The UI Backend is used for drawing basic widgets like combo boxes and windows, exposing a generic interface that is not platform-specific. This backend uses operating system user interface methods. Finally,

In [25]:
chat("What is a DNS lookup?")

'A DNS lookup is the process by which a computer translates a human-readable domain name into an IP address. Here are the steps involved in a DNS lookup:\n\n1. **User Request:** When you enter a URL in your browser, your operating system sends a request to the DNS resolver (usually provided by your operating system or web browser) to resolve the domain name.\n2. **Root DNS Server:** The DNS resolver starts at the root DNS server, which is like a phonebook for the internet. It asks the root server what the authoritative DNS server is for the top-level domain (TLD) of the URL you entered (e.g., .com).\n3. **TLD DNS Server:** The root DNS server directs the DNS resolver to the TLD DNS server. In this case, it would be the .com DNS server.\n4. **Authoritative DNS Server:** The TLD DNS server tells the DNS resolver where to find the authoritative DNS server for the specific domain name (e.g., example.com).\n5. **Zone File Query:** The authoritative DNS server checks its zone file, which con

In [27]:
print(chatHistory)



User : What is a browser's high level structure?
Assistant : The browser's high-level structure consists of several components:

1. **User Interface (UI):** This includes the address bar, back/forward button, bookmarking menu, and other visual elements that users interact with.
2. **Browser Engine:** This marshals actions between the UI and the rendering engine, ensuring a seamless interaction between these two components.
3. **Rendering Engine:** Responsible for displaying requested content, including parsing HTML and CSS, and displaying the parsed content on the screen.
4. **Networking:** Handles network calls such as HTTP requests, using different implementations for different platforms (behind a platform-independent interface).
5. **UI Backend:** Used for drawing basic widgets like combo boxes and windows, exposing a generic interface that is not platform-specific. This backend uses operating system user interface methods.
6. **JavaScript Engine:** Interpreter used to parse and e

In [28]:
chat("can you give me the ansewer to my last question as a paragraph?")

'Here is the answer to your previous question in a paragraph:\n\nA DNS lookup is the process by which a computer translates a human-readable domain name into an IP address. When you enter a URL in your browser, your operating system sends a request to the DNS resolver to resolve the domain name. The DNS resolver starts at the root DNS server, asking what the authoritative DNS server is for the top-level domain of the URL you entered. From there, it follows a series of steps to find the authoritative DNS server for the specific domain name and retrieve the associated IP address. This process involves checking zone files and caching responses so that subsequent lookups can be resolved more quickly.'

In [29]:
print(chatHistory)



User : What is a browser's high level structure?
Assistant : The browser's high-level structure consists of several components:

1. **User Interface (UI):** This includes the address bar, back/forward button, bookmarking menu, and other visual elements that users interact with.
2. **Browser Engine:** This marshals actions between the UI and the rendering engine, ensuring a seamless interaction between these two components.
3. **Rendering Engine:** Responsible for displaying requested content, including parsing HTML and CSS, and displaying the parsed content on the screen.
4. **Networking:** Handles network calls such as HTTP requests, using different implementations for different platforms (behind a platform-independent interface).
5. **UI Backend:** Used for drawing basic widgets like combo boxes and windows, exposing a generic interface that is not platform-specific. This backend uses operating system user interface methods.
6. **JavaScript Engine:** Interpreter used to parse and e